In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.stats import norm
from scipy.optimize import newton
import scipy.optimize as opt
import math
from mpl_toolkits.mplot3d import Axes3D

In [5]:


# Heston model parameters
S0 = 100  # Initial stock price
K = 100   # Strike price
r = 0.03  # Risk-free interest rate
T = 1.0   # Time to maturity
v0 = 0.04 # Initial volatility
kappa = 2.0
theta = 0.04
eta = 0.2
rho = 0.8

# Numerical parameters
x_max = 1.5  # Max x value for integration
N = 100      # Number of intervals for numerical integration


In [9]:
# Define the characteristic function of the Heston model
def characteristic_function(u, x, v, tau):
    alpha = -u**2 / 2 - 1j * u / 2
    beta = kappa - rho * eta * 1j * u
    gamma = 0.5 * eta**2
    
    d = np.sqrt(beta**2 - 4 * alpha * gamma)
    r_plus = (beta + d) / eta**2
    r_minus = (beta - d) / eta**2
    g = r_minus / r_plus
    
    C_tau = (r_minus * tau - 2 / eta**2) * np.log((1 - (1 / g) * np.exp(-d * tau)) / (1 - g))
    D_tau = (r_minus / (1 - np.exp(-d * tau))) * (1 - (1 / g) * np.exp(-d * tau))
    
    return np.exp(1j * u * x) * np.exp(C_tau * v + D_tau * v)



In [16]:
print(-x_max + 1 * 2 * x_max / N)

-1.47


In [10]:
# Define the numerical integration for P_j
def integrate_Pj(j, x, v, tau):
    integral = 0.0
    for n in range(N):
        u_n = -x_max + n * 2 * x_max / N
        integrand = (1 / (2 * np.pi)) * np.exp(-1j * u_n * x) * characteristic_function(u_n, x, v, tau) * (1j * u_n / 2)
        integral += integrand.real  # We take the real part
    return integral



In [11]:
# Calculate pseudo-probabilities P0 and P1 for European call option
def calculate_pseudo_probabilities(x, v, tau):
    P0 = 0.5 + integrate_Pj(0, x, v, tau)
    P1 = 0.5 + integrate_Pj(1, x, v, tau)
    return P0, P1



In [12]:
# Example usage
x_val = np.log(S0 / K)
P0_result, P1_result = calculate_pseudo_probabilities(x_val, v0, T)
print("P0:", P0_result)
print("P1:", P1_result)


P0: nan
P1: nan


C:\Users\Souvik Chakraborty\AppData\Local\Temp\ipykernel_11004\3460898696.py:12: RuntimeWarning: divide by zero encountered in cdouble_scalars
  C_tau = (r_minus * tau - 2 / eta**2) * np.log((1 - (1 / g) * np.exp(-d * tau)) / (1 - g))
C:\Users\Souvik Chakraborty\AppData\Local\Temp\ipykernel_11004\3460898696.py:12: RuntimeWarning: invalid value encountered in cdouble_scalars
  C_tau = (r_minus * tau - 2 / eta**2) * np.log((1 - (1 / g) * np.exp(-d * tau)) / (1 - g))
C:\Users\Souvik Chakraborty\AppData\Local\Temp\ipykernel_11004\3460898696.py:12: RuntimeWarning: invalid value encountered in log
  C_tau = (r_minus * tau - 2 / eta**2) * np.log((1 - (1 / g) * np.exp(-d * tau)) / (1 - g))
C:\Users\Souvik Chakraborty\AppData\Local\Temp\ipykernel_11004\3460898696.py:13: RuntimeWarning: divide by zero encountered in cdouble_scalars
  D_tau = (r_minus / (1 - np.exp(-d * tau))) * (1 - (1 / g) * np.exp(-d * tau))
C:\Users\Souvik Chakraborty\AppData\Local\Temp\ipykernel_11004\3460898696.py:13: Runti

In [17]:
print(x_val)

0.0


In [18]:
def PIntegrand(u, lambda_, vbar, eta, rho, v0, r, tau, S0, K, j):
        F = S0 * np.exp(r * tau)
        x = np.log(F / K)
        a = lambda_ * vbar

        if j == 1:
            b = lambda_ - rho * eta
            alpha = -u**2 / 2 - u / 2 * 1j + 1j * u
            beta = lambda_ - rho * eta - rho * eta * 1j * u
        else:
            b = lambda_
            alpha = -u**2 / 2 - u / 2 * 1j
            beta = lambda_ - rho * eta * 1j * u

        gamma = eta**2 / 2
        d = np.sqrt(beta**2 - 4 * alpha * gamma)
        rplus = (beta + d) / (2 * gamma)
        rminus = (beta - d) / (2 * gamma)
        g = rminus / rplus

        D = rminus * (1 - np.exp(-d * tau)) / (1 - g * np.exp(-d * tau))
        C = lambda_ * (rminus * tau - (2 / (eta**2)) * np.log((1 - g * np.exp(-d * tau)) / (1 - g)))

        top = np.exp(C * vbar + D * v0 + 1j * u * x)
        bottom = 1j * u
        real_part = (top / bottom).real
        return real_part

In [19]:
def P(lambda_, vbar, eta, rho, v0, r, tau, S0, K, j):
        value, _ = quad(PIntegrand, 0, np.inf, args=(lambda_, vbar, eta, rho, v0, r, tau, S0, K, j))
        prob = 0.5 + (1/np.pi) * value
        return prob

In [20]:
def HestonCallClosedForm(lambda_, vbar, eta, rho, v0, r, tau, S0, K):

    A = S0 * P(lambda_, vbar, eta, rho, v0, r, tau, S0, K, 1)
    B = K * np.exp(-r * tau) * P(lambda_, vbar, eta, rho, v0, r, tau, S0, K, 0)
    
    return A - B